#Загрузка необходимых библиотек

In [ ]:
import random as random 
import numpy as np 
import matplotlib.pyplot as plt # Импортируем pyplot 
import matplotlib.colors as colors # Импортируем colors
import pandas as pd
import time
import seaborn as sns
from IPython.display import display, HTML
sns.set_style('darkgrid')

#Функции

In [ ]:
#Функции для генетики
'''
    Функция получения выжившей популяции
        Входные параметры:
        - popul - наша популяция
        - val - текущие значения
        - nsurv - количество выживших
        - reverse - указываем требуемую операцию поиска результата: максимизация или минимизация
'''
def getSurvPopul(
        popul,
        val,
        nsurv,
        reverse
        ):
    newpopul = []                         # Двумерный массив для новой популяции
    sval = sorted(val, reverse=reverse)   # Сортируем зачения в val в зависимости от параметра reverse    
    for i in range(nsurv):                # Проходимся по циклу nsurv-раз (в итоге в newpopul запишется nsurv-лучших показателей)
        index = val.index(sval[i])        # Получаем индекс i-того элемента sval в исходном массиве val
        newpopul.append(popul[index])     # В новую папуляцию добавляем элемент из текущей популяции с найденным индексом
    return newpopul, sval                 # Возвращаем новую популяцию (из nsurv элементов) и сортированный список

'''
    Функция получения родителей
        Входные параметры:
        - curr_popul - текущая популяция
        - nsurv - количество выживших
'''
def getParents(
        curr_popul,
        nsurv
        ):   
    indexp1 = random.randint(0, nsurv - 1) # Случайный индекс первого родителя в диапазоне от 0 до nsurv - 1
    indexp2 = random.randint(0, nsurv - 1) # Случайный индекс второго родителя в диапазоне от 0 до nsurv - 1    
    botp1 = curr_popul[indexp1]            # Получаем первого бота-родителя по indexp1
    botp2 = curr_popul[indexp2]            # Получаем второго бота-родителя по indexp2    
    return botp1, botp2                    # Возвращаем обоих полученных ботов

'''
    Функция смешивания (кроссинговера) двух родителей
        Входные параметры:
        - botp1 - первый бот-родитель
        - botp2 - второй бот-родитель
        - j - номер компонента бота
'''
def crossPointFrom2Parents(
        botp1,
        botp2, 
        j
        ):
    pindex = random.random() # Получаем случайное число в диапазоне от 0 до 1
    
    # Если pindex меньше 0.5, то берем значения от первого бота, иначе от второго
    if pindex < 0.5:
        x = botp1[j]
    else:
        x = botp2[j]
    return x # Возвращаем значние бота

'''
    Функция для подсвечивания информации о пострианом мешке в DataFrame Pandas
'''
def закрасить_ячейку(дата_фрейм, номер_строки, номер_столбца, длительность, цвет_ячейки,  цвет_текста):
    color1 = 'background-color: green; color: white'
    color2 = 'background-color: yellow; color: black'
    df_styler = pd.DataFrame('', index=дата_фрейм.index, columns=дата_фрейм.columns)
    df_styler.iloc[номер_строки, номер_столбца] = color1
    for i in range(длительность):
      df_styler.iloc[номер_строки, номер_столбца-i-1] = color2
    return df_styler 

def info_bags(n=None):
  if n == None:
    print('Введите номер мешка, от', 0, 'до', len(количество_мешков)-1)
    n = int(input('Ввдите номер мешка: '))

  план = data_frame.iloc[:, 2:].to_numpy().reshape(-1)
  мешок_доступен = data_frame_m.iloc[1:, 1:].to_numpy().reshape(-1)

  for i in range(len(план)):    
    data = план[i].split(' ')    
    for j in data:      
      if str(n) + '(' in j:
        if j.index(str(n)+'(') == 0:
          result = i        
          break
  idx = result // 25
  col = ((result + время_на_стирку[idx] // 30) %25) + 2   

  длительность = время_на_стирку[idx] // 30
  if result%25 == 24:
    длительность = 0

  print('Номер машинки: ', idx + 1)
  print('Номер мешка: ', n)
  print('Всего мешков:', len(количество_мешков)-1, '\n')
  print('Время когда мешок готов к стирке:', мешок_доступен[n-1])
  print('Время начала стирки:', cycle_time[result %25])
  
  if длительность>0:
    print('Время готовности: ', cycle_time[result %25 + время_на_стирку[idx] // 30], '\n')
  print('Зеленым цветом выделено время, когда мешок будет постиран:',n)
  print('Желтым цветом выделено время, когда мешок стирается:',n, '\n')

  
  display(data_frame.style.apply(
            закрасить_ячейку,    
            номер_строки = idx,
            номер_столбца = col,
            длительность = время_на_стирку[idx] // 30,
            цвет_ячейки = 'green',
            цвет_текста = 'white',
            axis = None))
  print('Информация о мешках и времени когда мешок будет готов к стирке')
  display(data_frame_m.head(10))

#Подготовка данных

In [ ]:
cycle_time = ['8:00','8:30',	'9:00',	'9:30',	'10:00',	'10:30',	'11:00',	'11:30',	
            '12:00',	'12:30',	'13:00',	'13:30',	'14:00',	'14:30',	'15:00',	
            '15:30',	'16:00', '16:30',	'17:00',	'17:30',	'18:00',	'18:30',	
            '19:00',	'19:30',	'20:00']                              #Создаем временные метки с интервалом в 30-минут на одну смену с 8:00 - 20:00

rand_or_request = input('Ввести данные "случайно" или "вручную" ? ')  #Генерируем данные слчайно или вручную
rand_or_request = rand_or_request.lower()                             #Приводим запрос от пользователя в нижний регистр, чтобы при вводе можно было указать "случайно" или "вручную" с большой буквы

if all(i in 'случайно' for i in rand_or_request):                     #Если пользователь запросил случайную генерацию данных, то заранее напишем функцию которая будет возращять заранее подготовленные данные
  стиральные_машины = np.arange(1, 3+1)                               #Создаем массив, который включает в себя порядковый номер стиральной машины по количеству доступных стиральных машин

  вместимость_машин = []                                              #Пустой список, который будет включать в себя максимальную загрузку стиральных машин в "кг"
  for m in стиральные_машины:                                         #В цикле проходимся по каждой машине из массива стиральных машин
    вместимость_машин.append(random.randrange(10, 16, 2))              #Добавляем случайную вместимоть для каждой машины от 8-14 кг с шагом 2
  
  время_на_стирку = []                                                #Пустой список, который будет включать в себя цикл стрики для каждой машины от 60-120 минут с шагом в 30 минут
  for m in стиральные_машины:                                         #В цикле проходимся по каждой машине из массива стиральных машин
    время_на_стирку.append(random.randrange(60, 120, 30))             #Добавляем случайное время стирки от 60-120 минут с шагом в 30 минут
    
  мешки = 40                                                          #Количество мешков досупных утром
  количество_мешков = np.arange(мешки)                                #Создаем массив, где будут храниться порядковый номер каждого мешка  
  вес_мешков = np.random.randint(2, 12, len(количество_мешков))       #Масса каждого мешка от 2-12 "кг"
  
  новые_мешки = ['10:00', '15:00', '18:00']                           #Время когда приедут новые мешки
  
  количество_новых_мешков = np.arange(10)                             #Общее количество новых мешков   
  вес_новых_мешков =  np.random.randint(2, 12, len(количество_новых_мешков)) #Случайный вес для каждого мешка

  #добавляем новые мешки в конец общего массива 
  for i in range(len(количество_новых_мешков)):                       #Проходимся по количествую новых мешков
    id_maker = мешки + количество_новых_мешков[i]                     #Создаем новый порядковый номер для очередного нового мешка
    количество_мешков = np.append(количество_мешков, id_maker)        #Добавляем новые мешки в новым id у конец общего массива
  
  #Добавляем в конец новые веса, для новых мешков
  for i in range(len(вес_новых_мешков)):                              #Проходимся по размерности массива с весами для новых мешков
    вес_мешков = np.append(вес_мешков, вес_новых_мешков[i])           #Добавляем вес для новых мешков в конец массива с весами

  time = []                                                           #Список, где будет храниться время доступности мешков
  for i in range(len(количество_мешков)):                             #Добавляем начало смены в time, где будет храниться массив врмени когда мешок готов к стирке
    time.append(cycle_time[0])                                        #Добавляем в time 8:00 для мешков которые доступны к стирке в начале смены

  firs_bags_new = количество_новых_мешков[4]                          # 4 мешка приедет в '10:00'
  second_bags_new = количество_новых_мешков[3]                        # 3 мешка приедет в '15:00'
  third_bags_new = количество_новых_мешков[3]                         # 3 мешка приедет в '18:00'

  time_new_bags = []                                                  #Список где будет храниться время, когда приедет каждый новый мешок

  for t in range(third_bags_new):
      time_new_bags.append(новые_мешки[0])                            # Для каждого из 4 мешков добавим время готовности к стирке '10:00'
  for t in range(second_bags_new):
      time_new_bags.append(новые_мешки[1])                            # Для каждого из 3 мешков добавим время готовности к стирке '15:00'
  for t in range(firs_bags_new):
      time_new_bags.append(новые_мешки[2])                            # Для каждого из 3 мешков добавим время готовности к стирке '18:00'
  time[мешки:] = time_new_bags                                        #Добавляем время готовности к стирке новых мешков в общий список где будет хранитсья время готвности к стирке всех мешков

  #Создаем Data_Frame и заполним для каждой стиральной машинки максимальное количество "кг" которые они смогут постирать в зависимости от цикла стирки
  data_frame = pd.DataFrame({'Машинки': стиральные_машины, 'Макс. загрузка': вместимость_машин, 'Время цикла': время_на_стирку}) # Создаем колонки с информацией о стиральных машинках
  for t in cycle_time:
    data_frame[t] = 0                                                 #Создаем колонки с временными отрезками и заполняем нулями

  for m in range(len(стиральные_машины)):                             # Для каждой стиральной машины
    for t in range(len(cycle_time)):                                  # Для каждого врменного отрезка
      if t == 0:                                                      # 0 - это начало рабочего дня, поэтому каждая стиральная машина готова начать стирать
        data_frame.loc[(data_frame['Машинки'] == стиральные_машины[m]), '8:00'] = вместимость_машин[m] # По очереди добавляем в колонку 8:00 максимальную загрузку для каждой машинки вместимость_машин[m]
      
      else:                                                           #В остальных случаях, найдем такое время когда машинка закончила стирку и готова к новому циклу стирки
        df_mach = data_frame[(data_frame['Машинки'] == стиральные_машины[m])] #Промежуточный Data_frame, фильтруем по машинке для 1, 2, 3
        time_max = df_mach.values[:,2][0]                             #Берем информацию о том сколько машинка стирает
        cycle = time_max / 30                                         #Делим на 30 чтобы получить через какое количество временных отрезков мы получим время готовности к новой стирке
        available_time = np.arange(int(cycle), len(cycle_time), int(cycle)) # Создаем массив по количеству временых отрезков с шагом который мы получили в переменной cycle

        for av in available_time:                                     #Проходимся по каждой метке в массиве 
          data_frame.loc[(data_frame['Машинки'] == стиральные_машины[m]), cycle_time[av]] = вместимость_машин[m] #Добавляем к времени окончания цикла, количество "кг" котрое машинка готова загрузить

  machines = pd.DataFrame({'Машинки': стиральные_машины, 'Вместимость(кг)': вместимость_машин, 'Время на один цикл': время_на_стирку}) #Dara Drame с информацией о стиральных машинках
  data_frame_m = pd.DataFrame(data = [вес_мешков, time], columns=количество_мешков, index=['вес мешка', 'время доступа']) #Dara Drame с информацией мешках

  machines = machines.set_index('Машинки').dropna()                   #Установим порядковый номер машинки как индекс
  data_frame = data_frame.set_index('Машинки')                        #Установим порядковый номер машинки как индекс
  
  #Вектор временных индексов
  numpy_df = data_frame.to_numpy()                                    #переводим data_frame в numpy
  вместимость_на_день = numpy_df[:, 2:27]                             #Получаем значения с максимальной возможной загрузкой в "кг" для каждого времени
  вектор_вместимости_на_день = вместимость_на_день.reshape(len(стиральные_машины)*25,) #Разворачиваем массив из 3 на 25, в вектор 75 Где первые 25 элементов это загрузка для машинки №1 и с 25 - 50 для машинки №2 с 50-75 для машинки №3 

  '''
  Принты
  '''
  print('__________________________________________________________')
  print('Количество стиральных машинок:', len(стиральные_машины))
  print('Время цикла на стирку:', время_на_стирку)
  print('Вместимость:', вместимость_машин, '\n')

  print('Количество мешков в начале смены:', мешки)
  print('Количество новых мешков:', len(количество_мешков[мешки:]))
  print('Количество мешков на весь день:', len(количество_мешков))
  print('План на день:', вес_мешков.sum(), 'кг')
  print('Максимальная возможная загрузка на день:', вектор_вместимости_на_день.sum(),'кг')
  print()

  print('Информация о стиральных машинах')
  display(machines.head())
  print()
  print('Информация о мешках и времени когда мешок будет готов к стирке')
  display(data_frame_m.head())
  print()
  print('Информация о готовности машинок к стирке')
  display(data_frame.head())
  
  '''

  Ввод данных о прачечной вручную

  '''

  #Ввод от пользователя
elif all(i in 'вручную' for i in rand_or_request):                              #Если пользователь запросил ручной ввод данных, то запросим эти данные через input

  n_m = int(input('Введите количество стиральных машин: '))                     #Количество стиральных машин
  print()
  стиральные_машины = np.arange(n_m+1)                                          #Создаем массив по количеству стиральныхмашин, чтобы у каждой машинки был свой id

  вместимость_машин = []                                                        #Список для максимальной загрузки в "кг"
  for m in стиральные_машины:                                                   #Для каждой стиральной машинки
    print('Максимальная загрузка в "кг" для стиральной машинки под номером:', m)
    n_temp = int(input('Введите вместимость в "кг": '))                         #Запрашиваем у пользователя максимальную вместимость в стиральную машинку
    print()
    вместимость_машин.append(n_temp)                                            #Добавляем в список

  #Вводим максимальное время цикла стирки для каждой машины
  время_на_стирку = []                                                          #Время на цикл для каждой машинки
  for m in стиральные_машины:                                                   #Для каждой стиральной машинки
    print('Введите время цикла для стиральной машинки под номером:', m)
    print('От "30" - "720" минут, с интервалом в 30 минут' )
    print('Например 30 будет означать что машинка стирает "30" минут' )
    cycle = int(input('Время цикла стирки: '))                                  #Запрашиваем у пользователя максимальное время на один цикл
    print()
    время_на_стирку.append(cycle)                                               #Добавляем в список

  #Количесвто мешков, количество которое известно в начале смены                
  print('Введите количество мешков которые будут доступны к стирке,')
  print('cначала открытия рабочей смены:')
  мешки = int(input('Общее количество мешков:'))                                #Запрашиваем у пользователя количество мешков для стирки которое будет доступно к стирке в начале смены

  print()
  количество_мешков = np.arange(мешки)                                          #Создаем массив по количеству мешков, так мы получим id каждого мешка

  вес_мешков = np.random.randint(4, 8, len(количество_мешков))                  #Создадим случайный вес для каждого мешка
  
  
  time = []                                                                     #Список, где будет храниться время доступности мешков
  for i in range(мешки):                                                        #Добавляем начало смены в time, где будет храниться массив врмени когда мешок готов к стирке
    time.append(cycle_time[0])                                                  #Добавляем в time 8:00 для мешков которые доступны к стирке в начале смены

  #Новые мешки
  print('В какое время приедут новые мешки?')
  print('Введите время в диапазоне с 8:00 - 20:00')
  print('C интервалом в 30 минут, например 10:00 10:30:')
  новые_мешки = input().split()                                                 #Запросим у пользователя информацию в какое время приедут новые мешки, вводим время через пробел
  print()

  количество_новых_мешков = []                                                  #Массив, в котором будет храниться количество мешков для каждого времени
  temp_count_bags = []                                                          #Список с количеством мешков которое приедет в каждое введеное врмея
  for i in range(len(новые_мешки)):                                             #Проходимся в цикле по каждому времени, когда приедут новые мешки
    print('Какое количество мешков приедет в: ', новые_мешки[i])                #Спрашиваем у пользователя, какое количествомешков приедет в время указанное пользователем ранее
    temp = int(input('Введите количество мешков, например 5:'))                 #В переменнлй temp сохраняется количество мешков которое приедет в течении дня
    temp_count_bags.append(temp)                                                #Добавляем количество новыз мешков в списов 
    количество_новых_мешков.append(np.arange(temp))                             #В переменную количество_новых_мешков добавляем массив с id для новых мешков
    print()

  time_new_bags = []                                                            #Массив в котором будет храниться врмея прибытия в прачечную каждого нового мешка
  for i in range(len(новые_мешки)):                                             #Для каждого времени, когда приедут новые мешки
    t = temp_count_bags[i]                                                      #Получаем количество мешков которое приедет в течении дня
    for j in range(t):                                                          #В цикле добавляем для каждого нового мешка время прибытия в прачечную
      time_new_bags.append(новые_мешки[i])

  time[мешки:] = time_new_bags                                                  #Добавляем время прибытия новых мешков в общий массив где у нас хранитсья время готовности мешка к стирке

  вес_новых_мешков = []                                                         #Вес для каждого нового мешка
  for nm in range(len(temp_count_bags)):                                        
    вес_новых_мешков.append(np.random.randint(4, 8, temp_count_bags[nm]))       #Добавляем рандомные веса, для каждого нового мешка

  #добавляем в конец новые мешки
  for i in range(len(количество_новых_мешков)):                                 #В общий массив где у нас храняться id мешков, добавим id новых мешков
    id_maker = (количество_мешков[-1] + (количество_новых_мешков[i] + 1))       #Создаем id для новых мешков, 
    количество_мешков = np.concatenate([количество_мешков, id_maker])           #Добавляем новые id для новых мешков в общий массив

  #добавляем в конец новые веса
  for i in range(len(вес_новых_мешков)):                                        #Проходомся по всем весам для новых мешков
    вес_мешков = np.concatenate([вес_мешков, вес_новых_мешков[i]])              #Добавляем веса для новых мшков 

  #Создаем Data_Frame и заполним для каждой стиральной машинки максимальное количество "кг" которые они смогут постирать в зависимости от цикла стирки
  data_frame = pd.DataFrame({'Машинки': стиральные_машины, 'Макс. загрузка': вместимость_машин, 'Время цикла': время_на_стирку}) # Создаем колонки с информацией о стиральных машинках
  for t in cycle_time:
    data_frame[t] = 0                                                           #Создаем колонки с временными отрезками и заполняем нулями

  for m in range(len(стиральные_машины)):                                       # Для каждой стиральной машины
    for t in range(len(cycle_time)):                                            # Для каждого врменного отрезка
      if t == 0:                                                                # 0 - это начало рабочего дня, поэтому каждая стиральная машина готова начать стирать
        data_frame.loc[(data_frame['Машинки'] == стиральные_машины[m]), '8:00'] = вместимость_машин[m] # По очереди добавляем в колонку 8:00 максимальную загрузку для каждой машинки вместимость_машин[m]
      
      else:                                                                     #В остальных случаях, найдем такое время когда машинка закончила стирку и готова к новому циклу стирки
        df_mach = data_frame[(data_frame['Машинки'] == стиральные_машины[m])]   #Промежуточный Data_frame, фильтруем по машинке для 1, 2, 3
        time_max = df_mach.values[:,2][0]                                       #Берем информацию о том сколько машинка стирает
        cycle = time_max / 30                                                   #Делим на 30 чтобы получить через какое количество временных отрезков мы получим время готовности к новой стирке
        available_time = np.arange(int(cycle), len(cycle_time), int(cycle))     #Создаем массив по количеству временых отрезков с шагом который мы получили в переменной cycle

        for av in available_time:                                               #Проходимся по каждой метке в массиве 
          data_frame.loc[(data_frame['Машинки'] == стиральные_машины[m]), cycle_time[av]] = вместимость_машин[m] #Добавляем к времени окончания цикла, количество "кг" котрое машинка готова загрузить
  

  machines = pd.DataFrame({'Машинки': стиральные_машины, 'Вместимость(кг)': вместимость_машин, 'Время на один цикл': время_на_стирку}) #Dara Drame с информацией о стиральных машинках
  data_frame_m = pd.DataFrame(data = [вес_мешков, time], columns=количество_мешков, index=['вес мешка', 'время доступа']) #Dara Drame с информацией мешках

  machines = machines.set_index('Машинки').dropna()                             #Установим порядковый номер машинки как индекс
  data_frame = data_frame.set_index('Машинки')                                  #Установим порядковый номер машинки как индекс
  
  #Вектор временных индексов
  numpy_df = data_frame.to_numpy()                                              #переводим data_frame в numpy
  вместимость_на_день = numpy_df[:, 2:27]                                       #Получаем значения с максимальной возможной загрузкой в "кг" для каждого времени
  вектор_вместимости_на_день = вместимость_на_день.reshape(len(стиральные_машины)*25,) #Разворачиваем массив в вектор

  '''
  Блок принтов
  '''
  print('__________________________________________________________')
  print('Количество стиральных машинок:', len(стиральные_машины))
  print('Время цикла на стирку:', время_на_стирку)
  print('Вместимость:', вместимость_машин, '\n')
  print('Количество мешков в начале смены:', мешки)
  print('Количество новых мешков:', len(количество_мешков[мешки:]))
  print('Количество мешков на весь день:', len(количество_мешков))
  print('План на день:', вес_мешков.sum(), 'кг')
  print('Максимальная возможная загрузка на день:', вектор_вместимости_на_день.sum(),'кг')
  print()
  print('Информация о стиральных машинах')
  display(machines.head(10))
  print()
  print('Информация о мешках и времени когда мешок будет готов к стирке')
  display(data_frame_m.head(10))


Ввести данные "случайно" или "вручную" ? случайно
__________________________________________________________
Количество стиральных машинок: 3
Время цикла на стирку: [60, 90, 60]
Вместимость: [10, 10, 12] 

Количество мешков в начале смены: 40
Количество новых мешков: 10
Количество мешков на весь день: 50
План на день: 305 кг
Максимальная возможная загрузка на день: 376 кг

Информация о стиральных машинах


,Вместимость(кг),Время на один цикл
Машинки,,
1,10,60
2,10,90
3,12,60



Информация о мешках и времени когда мешок будет готов к стирке


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
вес мешка,10,5,5,7,6,8,6,2,7,5,7,9,10,6,8,9,2,4,2,11,9,7,7,3,11,3,8,3,3,4,3,10,2,2,4,9,4,8,9,7,7,5,11,5,8,2,3,10,3,6
время доступа,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,10:00,10:00,10:00,15:00,15:00,15:00,18:00,18:00,18:00,18:00



Информация о готовности машинок к стирке


,Макс. загрузка,Время цикла,8:00,8:30,9:00,9:30,10:00,10:30,11:00,11:30,12:00,12:30,13:00,13:30,14:00,14:30,15:00,15:30,16:00,16:30,17:00,17:30,18:00,18:30,19:00,19:30,20:00
Машинки,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,10,60,10,0,10,0,10,0,10,0,10,0,10,0,10,0,10,0,10,0,10,0,10,0,10,0,10
2,10,90,10,0,0,10,0,0,10,0,0,10,0,0,10,0,0,10,0,0,10,0,0,10,0,0,10
3,12,60,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12


In [ ]:
data_frame.head(10)

,Макс. загрузка,Время цикла,8:00,8:30,9:00,9:30,10:00,10:30,11:00,11:30,12:00,12:30,13:00,13:30,14:00,14:30,15:00,15:30,16:00,16:30,17:00,17:30,18:00,18:30,19:00,19:30,20:00
Машинки,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,10,60,10,0,10,0,10,0,10,0,10,0,10,0,10,0,10,0,10,0,10,0,10,0,10,0,10
2,10,90,10,0,0,10,0,0,10,0,0,10,0,0,10,0,0,10,0,0,10,0,0,10,0,0,10
3,12,60,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12


#Генетический алгоритм для рассчета плана на день

In [ ]:
import time
n = 100 # Общее число ботов
nsurv = 20 # Количество выживших (столько лучших переходит в новую популяцию)
nnew = n - nsurv  # Количество новых (столько новых ботов создается)
epohs = 1500 # количество эпох
mut = 0.3 # коэфициент мутаций

# Длина бота(количество мешков)
l = вес_мешков.shape[0] #Количество мешков которые надо распределить по машинкам
machines = вектор_вместимости_на_день.shape[0] #Получаем размер по всем временным метка для всех машинок
last_cycle = machines//25 
new_bags = вес_мешков[мешки:]
popul = [] # Двумерный массив популяции, размерностью [n, l].
val = [] # Одномерный массив значений этих ботов

plotmeanval = [] # сюда будут заносится значения для графика по среднему значению
plotminval = [] # сюда будут заносится значения для графика по минимальному значению

for i in range(n): # Проходим по всей длине популяции
    popul.append([]) # Создаем пустого бота
    for j in range(l): # Проходим по всей длине бота
        
        # В каждый компонент бота записываем рандомное значение в диапазоне от 0 до количества циклов на день
        popul[i].append(random.randint(0, machines - 1)) 

curr_time = time.time()

for it in range(epohs): # Проходим по всем эпохам
    
    if (it == 350): # Меняем коэфициент мутации после 100-ой эпохи
        mut = 0.1   
    if (it == 750): # Меняем коэфициет мутации после 250-ой эпохи
        mut = 0.05  
    if (it == 1200): # Меняем коэфициет мутации после 400-ой эпохи
        mut = 0.02 

    val = [] # Создаем пустой список для значений ботов
    for i in range(n): # Проходим по всей популяции
        bot = popul[i] # Берем очередного бота
        machinesfill = np.zeros(shape=machines) # Массив, хранящий заполняемость каждой машинки во время цикла
        for j in range(l): # Проходим по всей длине бота
            machinesfill[bot[j]] += вес_мешков[j] # Увеличиваем заполненность bot[j] на вес_мешков[j]          
        
        f = 0 # Обнуляем ошибку i-го бота 
        for t in range(machines): # Проходим по всем меткам циклов стирки
            # Увеличиваем ошибку i-го бота на модуль разницы между реальной вместимостью во время цикла 
            # и вместимостью, который посчитал бот
            f += abs(вектор_вместимости_на_день[t] - machinesfill[t])
  
        for i in range(1, len(стиральные_машины)+1):    #Функция ошибки для последнего цикла стирки
          time_error = i*25                 #Находим в векторе время последенего цикла стирки для каждой машинки
          if machinesfill[time_error-1] != 0:  #Поверяем чтобы предпоследний цикл для каждой машинки небыл занят, так как не успеет достирать 
            f += 100

        for b in range(len(bot)):           #Проходимся по длине пота
            if cycle_time.index(cycle_time[bot[b]%25]) < cycle_time.index(data_frame_m.iloc[1,b]):
              f+=100         
        val.append(f) # Добавляем в val значение ошибки для i-го бота    
    
    newpopul, sval = getSurvPopul(popul, val, nsurv, 0) # Получаем новую популяцию и сортированный список значнией
    print('Выполняется эпоха №' '{:>2} {:>2}'.format(it, ''), 
          '{:>3}'.format(np.round(time.time() - curr_time, 1)), 
          'сек,', '{:>15}'.format('10 лучших ботов'), 
          np.array2string(np.round(sval[0:10], 2).astype('int'),  
                          precision=2,suppress_small=True))   
    curr_time = time.time() # Обновляем текущее время
    plotmeanval.append(sum(val) / len(val)) # Добавляем среднее значение в список
    plotminval.append(sval[0]) # Добавляем минимальное значение в список
    
    for i in range(nnew): # Проходимся в цикле nnew-раз
        botp1, botp2 = getParents(newpopul, nsurv) # Из newpopul(новой популяции) получаем двух случайных родителей-ботов
        newbot = [] # Массив для нового бота
    
        for j in range(l): # Проходим по всей длине бота
            x = crossPointFrom2Parents(botp1, botp2, j) # Получаем значение для j-ого компонента бота
        
            # С вероятностью mut сбрасываем значение j-ого компонента бота на случайное
            if (random.random() < mut):
                x = random.randint(0, machines - 1)        
            newbot.append(x) # Добавляем новое значение в бота      
        newpopul.append(newbot) # Добавляем бота в новую популяцию    
    popul = newpopul # Записываем в popul новую посчитанную популяцию

# построение графиков 
plt.plot(plotmeanval, 
        label='Среднее по популяции')
plt.plot(plotminval, 
        label='Лучший бот')
plt.xlabel('Эпоха обучения')
plt.ylabel('Значение функции')
plt.legend()
plt.show()

bot = popul[0] # Берем лучшее значение в популяции
print('Значения лучшего бота:', bot, '\n') # Выводим значения бота

machinesfill = np.zeros(machines, dtype = 'int32') # Массив заполненности машинок
for j in range(l): # Проходим по всей длине бота
    machinesfill[bot[j]] += вес_мешков[j] # Увеличиваем заполненность bot[j]-ого поезда на size[j]вес_мешков

print('Максимальная возможная загрузка на день:', вектор_вместимости_на_день.sum(),'кг', '\n')
print('Плановое количество:', вес_мешков.sum(), 'кг')
print('Постирано:', machinesfill.sum(), 'кг')


machinesfill = np.full(len(вектор_вместимости_на_день), '', dtype='<U150') # Массив заполненности стиральных машин

for j in range(l): # Проходим по всей длине бота
  machinesfill[bot[j]] += str(j) + '(' + str(вес_мешков[j]) + ') ' #Добавляем в ячейку номер мешка(кг)
machinesfill_resh = machinesfill.reshape(стиральные_машины.shape[0], 25) #Переводим вектор в массива (количество машинок, и количество временных меток с 8:00 - 20:00, 25)

for m in range(len(machinesfill_resh)): #Для каждой стиральной машинки 
  for t in range(len(cycle_time)):  
    data_frame.loc[(data_frame.index == стиральные_машины[m]), cycle_time[t]] = machinesfill_resh[m][t] #Зполняем расписание, когда?Где?Какой? мешок постирался

#Посмотреть план на день

In [ ]:
data_frame.head(10)

,Макс. загрузка,Время цикла,8:00,8:30,9:00,9:30,10:00,10:30,11:00,11:30,12:00,12:30,13:00,13:30,14:00,14:30,15:00,15:30,16:00,16:30,17:00,17:30,18:00,18:30,19:00,19:30,20:00
Машинки,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6,120,21(4) 64(4),,,,4(4) 89(4),,,,42(7),,,,26(6),,,,123(6) 126(4),,,,114(6),,,,
1,7,60,88(6),,11(6),,8(7),,79(7),,94(7),,73(7),,70(5) 102(5),,92(6),,58(5) 100(6),,124(7),,98(7),,84(6) 125(4),,
2,15,90,6(7) 54(5) 83(4),,,31(5) 57(7) 65(5),,,62(5) 87(5) 96(7),,,35(5) 44(4) 103(5),,,43(7) 59(7),,,24(5) 27(6) 34(6),,110(5),71(4) 78(5) 108(6),,,9(4) 23(5) 50(7),,,
3,6,150,85(7),,,,,77(6),,,,,56(7),,,,,5(6),117(5),,,,109(4) 116(7),,,,
4,12,30,14(5) 37(7),10(4) 17(6) 38(5),0(5) 36(4) 41(5),52(5) 61(5) 74(4),49(4) 53(7),3(7) 69(7),22(5) 68(7),30(6) 40(5),13(6) 33(4) 104(5),18(6) 46(5) 97(5),63(5) 99(6),28(4) 72(5) 80(4),55(4) 75(7),20(6) 66(4) 91(7),67(7) 90(6),29(5) 45(6),106(5) 107(5) 119(4),60(7) 86(5),39(7) 121(5),51(7) 113(7),95(6) 111(6),1(7) 118(4),7(4) 15(4) 120(4),32(5) 82(5) 115(4),
5,8,90,19(7),,,48(6) 81(4),,,12(7) 25(6),,,16(6) 105(5),,,2(5) 47(5),,,101(7),,,112(7),122(4),,76(5) 93(7),,,


#Посмотреть информацию о мешке

In [ ]:
info_bags(126)

Номер машинки:  1
Номер мешка:  126
Всего мешков: 126 

Время когда мешок готов к стирке: 16:00
Время начала стирки: 16:00
Время готовности:  18:00 

Зеленым цветом выделено время, когда мешок будет постиран: 126
Желтым цветом выделено время, когда мешок стирается: 126 



,Макс. загрузка,Время цикла,8:00,8:30,9:00,9:30,10:00,10:30,11:00,11:30,12:00,12:30,13:00,13:30,14:00,14:30,15:00,15:30,16:00,16:30,17:00,17:30,18:00,18:30,19:00,19:30,20:00
Машинки,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6,120,21(4) 64(4),,,,4(4) 89(4),,,,42(7),,,,26(6),,,,123(6) 126(4),,,,114(6),,,,
1,7,60,88(6),,11(6),,8(7),,79(7),,94(7),,73(7),,70(5) 102(5),,92(6),,58(5) 100(6),,124(7),,98(7),,84(6) 125(4),,
2,15,90,6(7) 54(5) 83(4),,,31(5) 57(7) 65(5),,,62(5) 87(5) 96(7),,,35(5) 44(4) 103(5),,,43(7) 59(7),,,24(5) 27(6) 34(6),,110(5),71(4) 78(5) 108(6),,,9(4) 23(5) 50(7),,,
3,6,150,85(7),,,,,77(6),,,,,56(7),,,,,5(6),117(5),,,,109(4) 116(7),,,,
4,12,30,14(5) 37(7),10(4) 17(6) 38(5),0(5) 36(4) 41(5),52(5) 61(5) 74(4),49(4) 53(7),3(7) 69(7),22(5) 68(7),30(6) 40(5),13(6) 33(4) 104(5),18(6) 46(5) 97(5),63(5) 99(6),28(4) 72(5) 80(4),55(4) 75(7),20(6) 66(4) 91(7),67(7) 90(6),29(5) 45(6),106(5) 107(5) 119(4),60(7) 86(5),39(7) 121(5),51(7) 113(7),95(6) 111(6),1(7) 118(4),7(4) 15(4) 120(4),32(5) 82(5) 115(4),
5,8,90,19(7),,,48(6) 81(4),,,12(7) 25(6),,,16(6) 105(5),,,2(5) 47(5),,,101(7),,,112(7),122(4),,76(5) 93(7),,,


Информация о мешках и времени когда мешок будет готов к стирке


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126
вес мешка,5,7,5,7,4,6,7,4,7,4,4,6,7,6,5,4,6,6,6,7,6,4,5,5,5,6,6,6,4,5,6,5,5,4,6,5,4,7,5,7,...,5,6,4,6,7,6,7,7,6,7,5,7,6,6,7,5,5,5,5,5,5,6,4,5,6,7,7,6,4,7,5,4,4,4,5,4,6,7,4,4
время доступа,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,8:00,...,8:00,8:00,8:00,11:00,11:00,11:00,11:00,11:00,11:00,11:00,12:00,12:00,12:00,12:00,12:00,12:00,12:00,12:00,12:00,12:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00,16:00
